# Install and import libraries

In [1]:
%pip install pydot
%pip install tensorflow
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.




In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from pathlib import Path

from constants import (
    DATA_INPUT_PATH,
    MODEL_PATH,
    METADATA_PATH,
)

CLASSES = {
    0: "resting",
    1: "palm up",
    2: "closed fist",
    3: "ok",
    4: "pointer finger",
    5: "peace",
    6: "shaa",
    7: "peace among worlds"
}

# Read the files in the data dir

In [3]:
# Read all of the files in the data folder
files_in_folder = Path(DATA_INPUT_PATH).glob("*.csv")

files = [x for x in files_in_folder]
print([file for file in files])

[WindowsPath('data/emg_gestures_data_20250219_122322.csv')]


## Convert .csv(s) to dataframes and concatenate

In [4]:
# Read the data from the files
dfs = []

for file in files:
    df = pd.read_csv(str(file))
    dfs.append(df)
        
# Convert the data to a DataFrame
df = pd.concat([x for x in dfs], axis=0)

columns = ["gesture_id", "s1", "s2", "s3", "s4", "s5", "s6", "s7", "s8"]
df = df[columns]

print(df.head())

# Before removing duplicates
print(f"Shape of dataframe before removing duplicates {df.shape}")
# Remove duplicates
df = df.drop_duplicates()
print(f"Shape of dataframe after removing duplicates {df.shape}")

   gesture_id   s1   s2   s3  s4   s5   s6   s7   s8
0           0  366  558  412  97  131  379  114  257
1           0  191  419  374  73   85  200   65  122
2           0  127  334  372  71   66  110   47   82
3           0  109  320  355  70   78   92   45   76
4           0   91  263  357  67   78   83   41   67
Shape of dataframe before removing duplicates (9830, 9)
Shape of dataframe after removing duplicates (9830, 9)


In [5]:
# Parameters
window_size = 10  # Number of past samples to include

# Extract feature columns (excluding label, if it exists)
feature_columns = [col for col in df.columns if col != 'Label']
features = df[feature_columns].to_numpy()

# Convert features into a 3D array: [samples, features, time]
n_samples = len(features)
n_features = len(feature_columns)
reshaped_data = []

for i in range(n_samples):
    # Get the last `window_size` rows or fewer for each sample
    window = features[max(i - window_size + 1, 0):i + 1]
    
    # Pad the window if it has fewer than `window_size` rows
    if len(window) < window_size:
        padding = np.zeros((window_size - len(window), n_features))
        window = np.vstack((padding, window))
    
    reshaped_data.append(window)

# Convert list to 3D numpy array
reshaped_data = np.stack(reshaped_data, axis=0)
reshaped_data = reshaped_data.transpose(0, 2, 1)
df[feature_columns] = np.sqrt(np.mean(np.square(reshaped_data), axis=2))
    
print(df.shape)
print(df.head())

(9830, 9)
   gesture_id          s1          s2          s3         s4         s5  \
0         0.0  115.739362  176.455093  130.285840  30.674093  41.425837   
1         0.0  130.551522  220.663771  175.960223  38.390103  49.382183   
2         0.0  136.589165  244.638713  211.661050  44.473588  53.611566   
3         0.0  140.871218  264.741572  239.589023  49.677963  59.013558   
4         0.0  143.780388  277.497748  264.854300  54.007407  63.960926   

           s6         s7         s8  
0  119.850323  36.049965  81.270536  
1  135.514206  41.498193  89.962770  
2  139.907469  44.079474  93.625317  
3  142.900315  46.319542  96.660747  
4  145.290743  48.099896  98.955546  


## Scale and clean the data, then train the model

In [6]:
X = df.drop(columns=['gesture_id'])
y = df['gesture_id']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape the data for Conv1D
X_train_scaled = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_scaled = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

# # filter for valid classes because the data is not clean
# valid_classes = [0, 2, 3]
# mask_train = y_train.isin(valid_classes)
# mask_test = y_test.isin(valid_classes)

# # apply the mask to the training and testing data
# X_train_filtered = X_train_scaled[mask_train]
# y_train_filtered = y_train[mask_train]

# X_test_filtered = X_test_scaled[mask_test]
# y_test_filtered = y_test[mask_test]

# one hot encode the target data
y_train_categorical = to_categorical(y_train, num_classes=8)
y_test_categorical = to_categorical(y_test, num_classes=8)

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(8, activation='softmax')
])

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Compile the model with categorical crossentropy for multi-class classification
model.compile(
    optimizer=Adam(learning_rate=0.001), 
    loss='categorical_crossentropy', 
    metrics=['accuracy'],
)

# Train the model using the filtered training data
model.fit(
    X_train_scaled, 
    y_train_categorical, 
    epochs=100, 
    batch_size=64, 
    validation_split=0.2, 
    callbacks=[early_stopping]
)

# Evaluate the model on the filtered test set
test_loss, test_acc = model.evaluate(X_test_scaled, y_test_categorical)

print(f"Test accuracy: {test_acc}")

c:\Users\simpl\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.5885 - loss: 1.3329 - val_accuracy: 0.9650 - val_loss: 0.7917
Epoch 2/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9720 - loss: 0.1486 - val_accuracy: 0.9841 - val_loss: 0.2439
Epoch 3/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9824 - loss: 0.0867 - val_accuracy: 0.9886 - val_loss: 0.0714
Epoch 4/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9848 - loss: 0.0720 - val_accuracy: 0.9911 - val_loss: 0.0320
Epoch 5/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9862 - loss: 0.0554 - val_accuracy: 0.9949 - val_loss: 0.0194
Epoch 6/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9878 - loss: 0.0542 - val_accuracy: 0.9962 - val_loss: 0.0153
Epoch 7/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9908 - loss: 0.0381 - val_accuracy: 0.9955 - val_loss: 0.0135
Epoch 8/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9908 - loss: 0.0431 - val_accuracy: 0.9981 - v

In [7]:
# Save the model
model.save(f"model/{MODEL_PATH}")

# Save the scaler and the column names to a pickle file
with open(f"model/{METADATA_PATH}", 'wb') as f:
    pickle.dump((scaler, X_train.columns), f)

In [8]:
# thumbs-up then peace-sign
emg_data = [
    [160,245,126,32,28,25,26,99], # thumbs-up
    [67,197,559,104,41,82,257,169], # peace-sign
    [205,440,165,40,27,83,229,226], # gun-fingers
    [416,434,134,71,36,48,102,103] # fist
]

for data in emg_data:
    emg_features_df = pd.DataFrame([data], columns=X_train.columns)

    emg_features_scaled = scaler.transform(emg_features_df)
    emg_features_reshaped = emg_features_scaled.reshape(1, -1)

    prediction = model.predict(emg_features_reshaped)
    predicted_class = np.argmax(prediction)

    print(f"Predicted class: {predicted_class} - {CLASSES[predicted_class]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
Predicted class: 2 - closed fist
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Predicted class: 5 - peace
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Predicted class: 2 - closed fist
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Predicted class: 2 - closed fist
